# Scope
I receive a 140mb csv file daily into a share drive that I would like to use for historical analytics.  I have two problems I am trying to solve for:

    1. I want to create a historical parquet dataset as the storage for the historical analytics

    2. I want to know what combintation of tools I should use to perform the analytics

## Options
Problem 1

    1. Convert the daily file into its own parquet file and write that.  With that I can then use Dask or Pyarrow datasets
    
    2. Create partioned parquet dataset.  Partitioning strategy to be determined (i.e. weekly/monthly).  In particular partitioning is relevant because parquet does not have an append mode - so I cant simply append to the same parquet file or a file that contains a large number of dates.  As a result reading in the parquet file to memory to append the latest days data can get out of hand quickly (since I am appending all columns).  After a few months I will have an in memory dataframe of 12-16GB which will be very difficult for the PC and server we have to handle.  So a scalable solution is required. CAN PYARROW.CONCAT_TABLES HANDLE THIS?

Problem 2

    1. Pyarrow and pandas.  If were smart with data types and only choosing what columns we want this should suffice.
    2. Dask.
    3. Combination.  Dask for early steps, reading in and filtering, then convert to pandas when data is a managable size.

Data Source:
https://www.kaggle.com/benhamner/sf-bay-area-bike-share